In [256]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, confusion_matrix

from sklearn.utils import resample

In [2]:
df_cluster = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/clustered_data.csv', sep = ';')
df_cluster

,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Mes,Cluster
0,65.679216,735.558902,16.414147,19.501318,13.894332,116.433333,114.439583,2.144792,1.52,0,1,0
1,76.017708,734.376875,15.638204,18.355709,14.988855,40.879167,162.916667,1.103542,139.12,0,1,1
2,73.900711,733.959619,16.060072,18.713147,15.064426,59.225559,102.732787,1.328760,68.92,0,1,1
3,73.177223,734.321458,17.005510,19.832342,15.797179,82.889583,85.015766,1.597480,62.36,0,1,1
4,64.092344,736.860417,18.614403,21.592730,15.512999,161.429167,150.189583,2.222917,11.96,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3065,58.462548,924.237189,20.782350,24.296560,11.613997,86.694988,154.352083,8.483779,0.00,0,6,0
3066,78.492677,928.165211,19.225816,23.262681,15.223413,58.906011,125.716667,4.999583,0.32,0,6,0
3067,66.947345,928.851826,21.216116,24.902792,14.681483,82.298725,89.068750,7.688742,0.00,0,6,0
3068,54.981241,927.484876,21.871250,25.403125,11.656711,92.199144,143.372917,6.376733,0.00,0,6,0


In [213]:
xgb = xgboost.XGBClassifier()

cols_rem = ['Ordens', 'Cluster']

x = df_cluster[[c for c in df_cluster.columns if c not in cols_rem]]
y = df_cluster['Ordens']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378)

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':10, 'eta':2, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

In [214]:
print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
confusion_matrix(y_teste, y_teste_pred, normalize='true')

Treino: 0.8883201489064682
Teste: 0.8490770901194354
F1: 0.2871794871794872


array([[0.88290398, 0.11709602],
       [0.58208955, 0.41791045]])

In [189]:
df_cluster.groupby(['Ordens']).count()

,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Mes,Cluster
Ordens,,,,,,,,,,,
0,2846,2846,2846,2846,2846,2846,2846,2846,2846,2846,2846
1,224,224,224,224,224,224,224,224,224,224,224


In [402]:
df_clustered_total = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/clustered.csv', sep = ';')
df_clustered_total

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Cluster
0,2011-01-01 00:00:00,Camilopolis,84.8,920.6,20.200000,22.8,17.500000,0.000000,137.0,1.5,0.0,0,0
1,2011-01-01 00:00:00,RM,86.6,926.7,20.100000,23.3,17.800000,0.000000,131.0,2.5,0.0,0,0
2,2011-01-01 00:00:00,Paraiso,92.3,919.1,19.300000,22.7,18.000000,0.000000,101.0,2.6,0.0,0,0
3,2011-01-01 00:00:00,Erasmo,88.8,920.5,10.243265,22.9,17.700000,0.000000,133.0,0.7,0.0,0,0
4,2011-01-01 00:00:00,Vitoria,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2019-06-30 23:45:00,RM,66.9,931.9,18.800000,22.5,13.300000,0.294461,331.0,0.7,0.0,0,0
1463566,2019-06-30 23:45:00,Erasmo,60.7,925.7,19.500000,22.5,11.700000,0.000000,327.0,3.5,0.0,0,0
1463567,2019-06-30 23:45:00,Camilopolis,54.4,924.0,20.800000,23.0,12.200000,0.073893,332.0,4.3,0.0,0,0
1463568,2019-06-30 23:45:00,Paraiso,55.3,924.2,20.800000,22.1,11.790525,0.000000,25.0,3.6,0.0,0,0


In [403]:
df_clustered_total.groupby('Ordens').count()

,Data_Hora,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Cluster
Ordens,,,,,,,,,,,,
0,1426980,1426980,1426980,1426980,1426980,1426980,1426980,1426980,1426980,1426980,1426980,1426980
1,36590,36590,36590,36590,36590,36590,36590,36590,36590,36590,36590,36590


In [404]:
df_clustered_total['Local'] = df_clustered_total['Local'].rank(method='dense', ascending=False).astype(int)
df_clustered_total = df_clustered_total.drop(columns = 'Data_Hora')

In [405]:
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Cluster'].isin([1,2]))]
#df_slice = df_clustered_total[(df_clustered_total['Ordens'] == 1) | (df_clustered_total['Precipitacao'] > 10)]
df_slice = df_clustered_total.copy()
#df_slice.loc[df_slice['Cluster'] == 0, 'Ordens'] = 0
df_slice.loc[(df_slice['Ordens'] == 1) & (df_slice['Precipitacao'] <= 10), 'Ordens'] = 0

In [406]:
df_slice.groupby('Ordens').count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Cluster
Ordens,,,,,,,,,,,
0,1462584,1462584,1462584,1462584,1462584,1462584,1462584,1462584,1462584,1462584,1462584
1,986,986,986,986,986,986,986,986,986,986,986


In [407]:
df_slice

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Cluster
0,5,84.8,920.6,20.200000,22.8,17.500000,0.000000,137.0,1.5,0.0,0,0
1,2,86.6,926.7,20.100000,23.3,17.800000,0.000000,131.0,2.5,0.0,0,0
2,3,92.3,919.1,19.300000,22.7,18.000000,0.000000,101.0,2.6,0.0,0,0
3,4,88.8,920.5,10.243265,22.9,17.700000,0.000000,133.0,0.7,0.0,0,0
4,1,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2,66.9,931.9,18.800000,22.5,13.300000,0.294461,331.0,0.7,0.0,0,0
1463566,4,60.7,925.7,19.500000,22.5,11.700000,0.000000,327.0,3.5,0.0,0,0
1463567,5,54.4,924.0,20.800000,23.0,12.200000,0.073893,332.0,4.3,0.0,0,0
1463568,3,55.3,924.2,20.800000,22.1,11.790525,0.000000,25.0,3.6,0.0,0,0


In [408]:
xgb = xgboost.XGBClassifier()

cols_rem = ['Ordens', 'Cluster']

x = df_slice[[c for c in df_slice.columns if c not in cols_rem]]
#x = x.drop(columns = 'Cluster')
y = df_slice['Ordens']

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)

# concatenate our training data back together
X = pd.concat([x_treino, y_treino], axis=1)

# separate minority and majority classes
not_ordem = X[X['Ordens']==0].copy()
ordem = X[X['Ordens']==1].copy()

# upsample minority
ordem_upsampled = resample(ordem,
                        replace=True, # sample with replacement
                        n_samples=len(not_ordem), # match number in majority class
                        random_state=378) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_ordem, ordem_upsampled])

x_treino = upsampled[[c for c in df_slice.columns if c not in cols_rem]]
y_treino = upsampled['Ordens']

#xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino), (x_teste, y_teste)], eval_metric=f1_score)
param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

bst = xgboost.train(param, df_train, 2, feval=f1_score)
y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]

print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
print(f"F1: {f1_score(y_teste, y_teste_pred)}")
display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
display(confusion_matrix(y_teste, y_teste_pred,))

Treino: 0.9995160230081979
Teste: 0.9987405226034058
F1: 0.48172446110590444


array([[0.99882856, 0.00117144],
       [0.13175676, 0.86824324]])

array([[438261,    514],
       [    39,    257]], dtype=int64)

In [393]:
confusion_matrix(y_treino, y_treino_pred, )

# Treino: 0.9995160230081979
# Teste: 0.9987405226034058
# F1: 0.48172446110590444
# array([[0.99882856, 0.00117144],
#        [0.13175676, 0.86824324]])
# array([[438261,    514],
#        [    39,    257]], dtype=int64)

array([[1022818,     991],
       [      0, 1023809]], dtype=int64)

In [303]:
df_clustered_total

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Cluster
0,5,84.8,920.6,20.200000,22.8,17.500000,0.000000,137.0,1.5,0.0,0,0
1,2,86.6,926.7,20.100000,23.3,17.800000,0.000000,131.0,2.5,0.0,0,0
2,3,92.3,919.1,19.300000,22.7,18.000000,0.000000,101.0,2.6,0.0,0,0
3,4,88.8,920.5,10.243265,22.9,17.700000,0.000000,133.0,0.7,0.0,0,0
4,1,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1463565,2,66.9,931.9,18.800000,22.5,13.300000,0.294461,331.0,0.7,0.0,0,0
1463566,4,60.7,925.7,19.500000,22.5,11.700000,0.000000,327.0,3.5,0.0,0,0
1463567,5,54.4,924.0,20.800000,23.0,12.200000,0.073893,332.0,4.3,0.0,0,0
1463568,3,55.3,924.2,20.800000,22.1,11.790525,0.000000,25.0,3.6,0.0,0,0


In [252]:
df_slice.groupby(['Ordens']).count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Cluster
Ordens,,,,,,,,,,,
0,131202,131202,131202,131202,131202,131202,131202,131202,131202,131202,131202
1,36590,36590,36590,36590,36590,36590,36590,36590,36590,36590,36590


In [22]:
y_teste.shape[0] / y.shape[0]

0.300002383903881

In [23]:
df_slice

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Ordens,Cluster
480,3,94.400000,916.7,18.600000,21.900000,17.700000,0.0,161.0,3.4,0.0,0,1
481,2,91.300000,924.2,19.300000,22.222879,17.900000,0.0,160.0,2.6,0.0,0,1
482,5,92.500000,918.1,18.651871,21.800000,18.000000,0.0,188.0,1.6,0.2,0,1
483,4,94.400000,918.1,18.700000,21.749954,17.800000,0.0,118.0,3.3,0.2,0,1
484,1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1461627,2,82.800000,932.2,18.800000,21.934040,16.600000,0.0,5.0,2.2,0.0,1,0
1461631,1,89.243362,926.1,17.800000,25.000000,16.237339,0.0,0.0,0.0,0.0,1,0
1461636,4,88.200000,926.0,17.800000,20.900000,15.800000,0.0,166.0,5.0,0.0,1,0
1461640,4,88.400000,926.2,17.900000,21.000000,16.000000,0.0,221.0,7.2,0.0,1,0


In [62]:
# import pandas_gbq
# from google.oauth2 import service_account

# PROJECT_ID = 'temporal-285820'
# TABLE_clustered = 'info_pluviometrica.clustered_slice'

# CREDENTIALS = service_account.Credentials.from_service_account_file('../../../key/temporal-285820-cde76c259484.json')
# pandas_gbq.context.credentials = CREDENTIALS

In [63]:
# pandas_gbq.to_gbq(df_slice, TABLE_clustered, project_id=PROJECT_ID, credentials=CREDENTIALS, if_exists='replace')
# print('clustered done!')

1it [00:19, 19.36s/it]clustered done!



In [65]:
df_m = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/merged.csv', sep = ';')

In [74]:
df_clustered_total[(df_clustered_total['Cluster'].isin([1,2]))].groupby('Ordens').count()

,Local,UmidadeRelativa,PressaoAtmosferica,TemperaturaDoAr,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,Cluster
Ordens,,,,,,,,,,,
0,131202,131202,131202,131202,131202,131202,131202,131202,131202,131202,131202
1,19498,19498,19498,19498,19498,19498,19498,19498,19498,19498,19498
